In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import imageio
from IPython.display import Image as IPImage

# =========================
# 1️⃣ CREAR IMAGEN BASE
# =========================
img = np.zeros((800, 1000, 3), dtype=np.uint8)

# Colores brillantes para detección
COLOR_CIRCULOS = (255, 255, 100)    # Cian brillante
COLOR_RECTANGULOS = (100, 255, 100) # Verde claro
COLOR_TRIANGULOS = (100, 100, 255)  # Rojo/naranja

# ========================
# 🔵 CÍRCULOS (3 tamaños)
# ========================
cv2.circle(img, (150, 200), 60, COLOR_CIRCULOS, 2)   # mediano
cv2.circle(img, (400, 150), 90, COLOR_CIRCULOS, 2)   # grande
cv2.circle(img, (250, 400), 40, COLOR_CIRCULOS, 2)   # pequeño

# =========================
# ⬛ RECTÁNGULOS (3 tamaños)
# =========================
cv2.rectangle(img, (600, 100), (780, 250), COLOR_RECTANGULOS, 2)  # grande
cv2.rectangle(img, (550, 300), (650, 400), COLOR_RECTANGULOS, 2)  # mediano
cv2.rectangle(img, (700, 450), (780, 520), COLOR_RECTANGULOS, 2)  # pequeño

# =========================
# 🔺 TRIÁNGULOS (3 tamaños)
# =========================
pts1 = np.array([[150, 650], [250, 500], [350, 650]], np.int32)
cv2.polylines(img, [pts1], True, COLOR_TRIANGULOS, 2)

pts2 = np.array([[450, 700], [500, 600], [550, 700]], np.int32)
cv2.polylines(img, [pts2], True, COLOR_TRIANGULOS, 2)

pts3 = np.array([[700, 700], [730, 640], [760, 700]], np.int32)
cv2.polylines(img, [pts3], True, COLOR_TRIANGULOS, 2)

# =========================
# 2️⃣ PROCESAMIENTO
# =========================
# Convertir a escala de grises
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Umbralización con valor bajo para detectar todos los colores
_, thresh = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY)

# Detectar contornos
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

print(f"Contornos detectados: {len(contours)}")

# =========================
# 3️⃣ ANÁLISIS DE CONTORNOS
# =========================
img_propiedades = img.copy()

COLOR_CONTORNO = (0, 255, 0)      # Verde brillante
COLOR_CENTROIDE = (0, 0, 255)     # Rojo brillante
COLOR_TEXTO = (255, 255, 255)     # Blanco

for cnt in contours:
    area = cv2.contourArea(cnt)
    perimeter = cv2.arcLength(cnt, True)
    
    M = cv2.moments(cnt)
    if M["m00"] != 0:
        cx = int(M["m10"] / M["m00"])
        cy = int(M["m01"] / M["m00"])
    else:
        cx, cy = 0, 0
    
    cv2.drawContours(img_propiedades, [cnt], -1, COLOR_CONTORNO, 2)
    cv2.circle(img_propiedades, (cx, cy), 5, COLOR_CENTROIDE, -1)
    
    cv2.putText(img_propiedades, f"A={int(area)}", (cx-40, cy-10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLOR_TEXTO, 1)
    cv2.putText(img_propiedades, f"P={int(perimeter)}", (cx-40, cy+10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLOR_TEXTO, 1)

# =========================
# 4️⃣ CLASIFICACIÓN DE FORMAS
# =========================
img_clasificacion = img.copy()

COLOR_TRIANGULO = (0, 255, 255)   # Amarillo
COLOR_CUADRADO = (255, 0, 255)    # Magenta
COLOR_CIRCULO = (255, 255, 0)     # Cian

for cnt in contours:
    perimeter = cv2.arcLength(cnt, True)
    approx = cv2.approxPolyDP(cnt, 0.02 * perimeter, True)
    
    M = cv2.moments(cnt)
    if M["m00"] != 0:
        cx = int(M['m10'] / M['m00'])
        cy = int(M['m01'] / M['m00'])
    else:
        continue
    
    if len(approx) == 3:
        shape = "Triangulo"
        color = COLOR_TRIANGULO
    elif len(approx) == 4:
        shape = "Cuadrado"
        color = COLOR_CUADRADO
    else:
        shape = "Circulo"
        color = COLOR_CIRCULO
    
    cv2.drawContours(img_clasificacion, [cnt], -1, color, 3)
    cv2.putText(img_clasificacion, shape, (cx-50, cy+30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, COLOR_TEXTO, 2)

# =========================
# 5️⃣ GENERAR GIF MUY LENTO
# =========================
frames = []

# Frame 1: Imagen original
frame1 = img.copy()
cv2.putText(frame1, "Etapa 1: Imagen Original", (20, 40),
            cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 3)
cv2.putText(frame1, "Circulos (Cian) | Rectangulos (Verde) | Triangulos (Rojo)", 
            (20, 780), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
frames.append(cv2.cvtColor(frame1, cv2.COLOR_BGR2RGB))

# Frame 2: Escala de grises
frame2 = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)
cv2.putText(frame2, "Etapa 2: Escala de Grises", (20, 40),
            cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 3)
cv2.putText(frame2, "Conversion BGR -> Escala de Grises (1 canal)", 
            (20, 780), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
frames.append(cv2.cvtColor(frame2, cv2.COLOR_BGR2RGB))

# Frame 3: Binarización
frame3 = cv2.cvtColor(thresh, cv2.COLOR_GRAY2BGR)
cv2.putText(frame3, "Etapa 3: Binalizacion (Umbral=50)", (20, 40),
            cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 3)
cv2.putText(frame3, f"Contornos detectados: {len(contours)} figuras geometricas", 
            (20, 780), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
frames.append(cv2.cvtColor(frame3, cv2.COLOR_BGR2RGB))

# Frame 4: Detección de contornos
frame4 = img.copy()
cv2.drawContours(frame4, contours, -1, (0, 255, 0), 3)
cv2.putText(frame4, "Etapa 4: Deteccion de Contornos", (20, 40),
            cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 3)
cv2.putText(frame4, "Algoritmo: cv2.findContours() con modo RETR_EXTERNAL", 
            (20, 780), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
frames.append(cv2.cvtColor(frame4, cv2.COLOR_BGR2RGB))

# Frame 5: Propiedades
frame5 = img_propiedades.copy()
cv2.putText(frame5, "Etapa 5: Calculo de Propiedades", (20, 40),
            cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 3)
cv2.putText(frame5, "A=Area (pixeles²) | P=Perimetro (pixeles) | Punto rojo=Centroide", 
            (20, 780), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
frames.append(cv2.cvtColor(frame5, cv2.COLOR_BGR2RGB))

# Frame 6: Clasificación
frame6 = img_clasificacion.copy()
cv2.putText(frame6, "Etapa 6: Clasificacion de Formas", (20, 40),
            cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 3)
cv2.putText(frame6, "Metodo: Aproximacion poligonal (Douglas-Peucker) + conteo de vertices", 
            (20, 780), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
frames.append(cv2.cvtColor(frame6, cv2.COLOR_BGR2RGB))

# ⚠️ DURACIÓN MUY LENTA: 5 SEGUNDOS POR FRAME
imageio.mimsave('analisis.gif', frames, duration=200.0, loop=0)


# =========================
# 6️⃣ VISUALIZACIÓN FINAL
# =========================
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Pipeline Completo de Análisis Geométrico - OpenCV', 
             fontsize=18, fontweight='bold', y=0.98)

axes[0, 0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
axes[0, 0].set_title('1. Imagen Original\n(9 figuras con colores diferenciados)', 
                     fontweight='bold', fontsize=11)
axes[0, 0].axis('off')

axes[0, 1].imshow(gray, cmap='gray')
axes[0, 1].set_title('2. Escala de Grises\n(Reducción a 1 canal)', 
                     fontweight='bold', fontsize=11)
axes[0, 1].axis('off')

axes[0, 2].imshow(thresh, cmap='gray')
axes[0, 2].set_title(f'3. Binarización (Umbral=50)\n{len(contours)} contornos detectados', 
                     fontweight='bold', fontsize=11)
axes[0, 2].axis('off')

frame4_show = img.copy()
cv2.drawContours(frame4_show, contours, -1, (0, 255, 0), 3)
axes[1, 0].imshow(cv2.cvtColor(frame4_show, cv2.COLOR_BGR2RGB))
axes[1, 0].set_title('4. Contornos Detectados\n(Algoritmo RETR_EXTERNAL)', 
                     fontweight='bold', fontsize=11)
axes[1, 0].axis('off')

axes[1, 1].imshow(cv2.cvtColor(img_propiedades, cv2.COLOR_BGR2RGB))
axes[1, 1].set_title('5. Propiedades Geométricas\n(Área, Perímetro, Centroide)', 
                     fontweight='bold', fontsize=11)
axes[1, 1].axis('off')

axes[1, 2].imshow(cv2.cvtColor(img_clasificacion, cv2.COLOR_BGR2RGB))
axes[1, 2].set_title('6. Clasificación por Forma\n(Aproximación poligonal)', 
                     fontweight='bold', fontsize=11)
axes[1, 2].axis('off')

plt.tight_layout()
plt.savefig('pipeline_completo.png', dpi=150, bbox_inches='tight')
plt.show()


Contornos detectados: 9
